In [ ]:
# 2nd question

from flask import Flask, render_template, request
from pymongo import MongoClient

app = Flask(__name__)

# Connect to MongoDB running in the container (mongo service)
client = MongoClient("mongodb://mongo:27017/")
db = client["labdb"]
users_collection = db["users"]

@app.route("/")
def home():
    # Simple home page with links
    return "<h2>Welcome to the Lab Web App</h2><a href='/register'>Register</a> | <a href='/login'>Login</a>"

# Handle registration form    
  
@app.route('/register',methods=['GET','POST'])
def register():
	if request.method=='POST':
		username=request.form['username']
		password=request.form['password']

		users_collection.insert_one({
			'username':username,
			'password':password
		})
		return 'Registration Successful'
	return render_template('register.html')

@app.route('/login',methods=['GET','POST'])
def login():
	if request.method=='POST':
		username=request.form['username']
		password=request.form['password']
		user=users_collection.find_one({
			'username':username,
			'password':password
		})
		if user:
			return f'Welcome, {username}'
		else:
			return "Invalid"
	
	return render_template("login.html")

if __name__ == "__main__":
  app.run(host="0.0.0.0", debug=True)

In [ ]:
# 3rd question

from flask import Flask, request, render_template, send_from_directory, redirect, url_for
import os

app = Flask(__name__)

# Absolute path for reliability
UPLOAD_FOLDER = os.path.abspath("uploads")
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

# Create uploads folder if it does not exist
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)


# -----------------------------
# Home page -> upload form
# -----------------------------
@app.route("/")
def home():
    return render_template("upload.html")


# -----------------------------
# File Upload Functionality
# -----------------------------
@app.route("/upload", methods=["POST"])
def upload_file():
    # Read the uploaded file from request.files
    if "file" not in request.files:
        return "No file part in request"

    file = request.files["file"]

    if file.filename == "":
        return "No file selected"

    # Save the file into the uploads folder
    file_path = os.path.join(app.config["UPLOAD_FOLDER"], file.filename)
    file.save(file_path)

    # Render success page
    return render_template("upload_success.html", filename=file.filename)


# -----------------------------
# Create a default file if uploads folder is empty
# -----------------------------
default_file_path = os.path.join(UPLOAD_FOLDER, "sample.txt")

if not os.path.exists(default_file_path):
    with open(default_file_path, "w") as f:
        f.write("This is a sample file for download testing.")


# -----------------------------
# File Download Functionality
# -----------------------------
@app.route("/download/<filename>")
def download_file(filename):
    # Send the file for download
    return send_from_directory(
        app.config["UPLOAD_FOLDER"],
        filename,
        as_attachment=True
    )


# -----------------------------
# List Uploaded Files
# -----------------------------
@app.route("/files")
def list_files():
    files = os.listdir(app.config["UPLOAD_FOLDER"])
    return render_template("files.html", files=files)


if __name__ == "__main__":
    app.run(host="0.0.0.0", debug=True)

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Student Records</title>

    <!-- Link the external CSS file -->
    <link rel="stylesheet" type="text/css" href="/static/styles.css">
</head>
<body>

    <h2>MongoDB Student Records</h2>

    <table class="styled-table">
        <tr>
            <th>_id</th>
            <th>Name</th>
            <th>Email</th>
            <th>Age</th>
        </tr>

        <!-- Loop through MongoDB data -->
        {% for row in data %}
        <tr>
            <td>{{ row["_id"] }}</td>
            <td>{{ row["name"] }}</td>
            <td>{{ row["email"] }}</td>
            <td>{{ row["age"] }}</td>
        </tr>
        {% endfor %}
    </table>

</body>
</html>


4th question HTML Code

from flask import Flask, render_template
from pymongo import MongoClient

app = Flask(__name__)

# MongoDB Connection (already provided in your environment)
client = MongoClient("mongodb://mongo:27017/")   # Change only if needed
db = client["labdb"]                                 # Database name
collection = db["students"]                           # Collection name

@app.route("/viewdata")
def view_data():

    # i. Retrieve all documents from MongoDB
    cursor = collection.find()

    # ii. Convert cursor into a list
    data = list(cursor)

    # iii. Pass data to HTML template
    return render_template("table.html", data=data)


if __name__ == "__main__":
    app.run(host="0.0.0.0", debug=True)



In [ ]:
# 5th question

from flask import Flask, request, render_template

app = Flask(__name__)

# Route to display the form
@app.route("/")
def home():
    return render_template("form.html")

@app.route("/process", methods=["POST"])
def process():
    text = request.form["userdata"]
    result = text.upper()
    return result


if __name__ == "__main__":
    app.run(host="0.0.0.0", debug=True)